In [135]:
import pandas as pd
from pandasql import sqldf


def table_formater(df, seperator='|', col_data_split='', col_prefix='[HEAD]', row_prefix='[ROW]', permute_df=True, utterance=None):

    # if permute_df and utterance is not None:
    #     df = permuteDataFrame(df, utterance)['col reorder based on name overlaps']
    
    table_str = []
    if col_prefix != '':
        table_str.append(col_prefix + ': ' + seperator.join(df.columns.tolist()))
    else:
        table_str.append(seperator.join(df.columns.tolist()))
        
    if col_data_split != '':
        table_str.append(col_data_split * len(table_str[-1]))
    for i in range(df.shape[0]):
        if row_prefix != '':
            table_str.append(row_prefix + f' {i+1}: ' + seperator.join([str(i) for i in df.iloc[i].tolist()]))
        else:
            table_str.append(seperator.join([str(i) for i in df.iloc[i].tolist()]))
    return '\n'.join(table_str)



latex_special_token = ["!@#$%^&*()"]

def generate(text_list, attention_list, latex_file, color='red', rescale_value = False):
    assert(len(text_list) == len(attention_list))
    if rescale_value:
        attention_list = rescale(attention_list)
    word_num = len(text_list)
    text_list = clean_word(text_list)
    with open(latex_file,'w') as f:
        f.write(r'''\documentclass[varwidth]{standalone}
\special{papersize=210mm,297mm}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}'''+'\n')
        string = r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
        for idx in range(word_num):
            if '\n' in text_list[idx]:
                text_list[idx] = text_list[idx].replace('\n', '\\\\')#.replace('|', '\\|')
                suffix = '\\\\'
            else:
                suffix = ''
            string += "\\colorbox{%s!%s}{"%(color, attention_list[idx])+"\\strut " + text_list[idx]+"} " + suffix + ' '
        string += "\n}}}"
        f.write(string+'\n')
        f.write(r'''\end{CJK*}
\end{document}''')

def rescale(input_list, soft_max=False):
    if soft_max:
        x = np.array(input_list)
        rescale = np.exp(x)/np.exp(x).sum()
        scale_factor = 100 / np.max(rescale)
        rescale *= scale_factor      
    else:
        the_array = np.asarray(input_list)
        the_max = np.max(the_array)
        the_min = np.min(the_array)
        rescale = (the_array - the_min)/(the_max-the_min)
        scale_factor = 100 / np.max(rescale)
        rescale *= scale_factor      
    print("rescaled to max: ", np.max(rescale), ', min: ', np.min(rescale), ', sum: ', np.sum(rescale))
    return rescale.tolist()

def clean_word(word_list):
	new_word_list = []
	for word in word_list:
		for latex_sensitive in ["\\", "%", "&", "^", "#", "_",  "{", "}"]:
			if latex_sensitive in word:
				word = word.replace(latex_sensitive, '\\'+latex_sensitive)
		new_word_list.append(word)
	return new_word_list



df = pd.read_csv(f'csv/203-csv/585.csv', sep=',', on_bad_lines='warn')
# df.columns = ['Team', 'County', 'Wins', 'Years_won']
cols = df.columns.tolist()
cols

['Rank', 'Nation', 'Gold', 'Silver', 'Bronze', 'Total']

In [33]:
import openai
import dotenv
config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY_ms']
# prompt = f"""
# Answer the question based on the data below: "who had the most silver medals?". 

# A selected table: 
# {table_formater(concise_df)}

# The full table is:
# {table_formater(df)}

# The answer is: ```
# """

prompt = """
Answer the question based on the data below: "who had the most Silver medals?". 

[HEAD]: Rank|Nation|Gold|Silver|Bronze|Total
[ROW] 1: 1|Soviet Union|7|3|6|16
[ROW] 2: 2|Austria|4|3|4|11
[ROW] 3: 3|Finland|3|3|1|7
[ROW] 4: 4|Switzerland|3|2|1|6
[ROW] 5: 5|Sweden|2|4|4|10
[ROW] 6: 6|United States|2|3|2|7
[ROW] 7: 7|Norway|2|1|1|4
[ROW] 8: 8|Italy|1|2|0|3
[ROW] 9: 9|Germany|1|0|1|2
[ROW] 10: 10|Canada|0|1|2|3

The answer is: ```
"""
print(prompt)


Answer the question based on the data below: "who had the most Silver medals?". 

[HEAD]: Rank|Nation|Gold|Silver|Bronze|Total
[ROW] 1: 1|Soviet Union|7|3|6|16
[ROW] 2: 2|Austria|4|3|4|11
[ROW] 3: 3|Finland|3|3|1|7
[ROW] 4: 4|Switzerland|3|2|1|6
[ROW] 5: 5|Sweden|2|4|4|10
[ROW] 6: 6|United States|2|3|2|7
[ROW] 7: 7|Norway|2|1|1|4
[ROW] 8: 8|Italy|1|2|0|3
[ROW] 9: 9|Germany|1|0|1|2
[ROW] 10: 10|Canada|0|1|2|3

The answer is: ```



In [107]:
model = 'davinci-codex-002-msft'
# model = 'text-davinci-002'
# model = 'text-davinci-003'

openai.Completion.create(   engine=model,
                            prompt=prompt,
                            max_tokens=128,
                            temperature=0,
                            top_p=1,
                            frequency_penalty=0,
                            n=1,
                            stream=False,
                            stop='```')

<OpenAIObject text_completion id=cmpl-6PRJGDXCwOAim7skbcXyW8xiHs9bW at 0x7fb8e437bae0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "Soviet Union"
    }
  ],
  "created": 1671521198,
  "id": "cmpl-6PRJGDXCwOAim7skbcXyW8xiHs9bW",
  "model": "davinci-codex-002-msft",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 2,
    "prompt_tokens": 233,
    "total_tokens": 235
  }
}

In [41]:
tokenizer.encode('Austria')


[15160, 7496]

In [140]:
prompt = """
Answer the question based on the data below: "who had the most Silver medals?". 

[HEAD]: Rank|Nation|Gold|Silver|Bronze|Total
[ROW] 1: 1|Soviet Union|7|3|6|16
[ROW] 2: 2|Austria|4|3|4|11
[ROW] 3: 3|Finland|3|3|1|7
[ROW] 4: 4|Switzerland|3|2|1|6
[ROW] 5: 5|Sweden|2|4|4|10
[ROW] 6: 6|United States|2|3|2|7
[ROW] 7: 7|Norway|2|1|1|4
[ROW] 8: 8|Italy|1|2|0|3
[ROW] 9: 9|Germany|1|0|1|2
[ROW] 10: 10|Canada|0|1|2|3

The answer is: 
"""
# prompt = "Machine learning with PyTorch can do amazing"
# prompt = "Machine learning with PyTorch can do amazing"

from transformers import AutoTokenizer, AutoModel, utils, AutoModelForCausalLM
import torch
model = 'gpt2'
model = 't5-large'
utils.logging.set_verbosity_error()  # Suppress standard warnings
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModel.from_pretrained(model, output_attentions=True)

predicted = ''
word_cnt = 1
while word_cnt:
    ori_input_ids = tokenizer.encode(prompt, return_tensors='pt')
    outputs = model(ori_input_ids)
    attentions = outputs.attentions
    pred_id = torch.argmax(outputs.logits[:, -1, :]).item()
    pred_word = tokenizer.decode(pred_id)
    # print("Predict: \'", pred_word, "\'")
    predicted += pred_word
    prompt += pred_word
    word_cnt -= 1
    

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [85]:
attentions[-1].shape

torch.Size([1, 12, 9, 9])

In [12]:
outputs.logits.shape

torch.Size([1, 233, 50257])

In [45]:
inputs

tensor([[  198, 33706,   262,  1808,  1912,   319,   262,  1366,  2174,    25,
           366,  8727,   550,   262,   749,  7698, 28057, 43634,   220,   198,
           198,    58, 37682,  5974, 10916,    91, 46108,    91, 13306,    91,
         26766,    91, 18760,  2736,    91, 14957,   198,    58,    49,  3913,
            60,   352,    25,   352,    91, 40408,  4479,    91,    22,    91,
            18,    91,    21,    91,  1433,   198,    58,    49,  3913,    60,
           362,    25,   362,    91, 15160,  7496,    91,    19,    91,    18,
            91,    19,    91,  1157,   198,    58,    49,  3913,    60,   513,
            25,   513,    91, 18467,  1044,    91,    18,    91,    18,    91,
            16,    91,    22,   198,    58,    49,  3913,    60,   604,    25,
           604,    91, 10462, 13947,    91,    18,    91,    17,    91,    16,
            91,    21,   198,    58,    49,  3913,    60,   642,    25,   642,
            91, 10462, 31829,    91,    17,    91,  

'\n'

In [ ]:
from bertviz import head_view
head_view(attention, tokens)

In [148]:
import torch
t = torch.zeros([len(tokens), len(tokens)]).float()
for att in attention:
    t += torch.mean(torch.squeeze(att), axis=0)

In [184]:
outputs.last_hidden_state.shape, inputs.shape, 

(torch.Size([1, 63, 768]), torch.Size([1, 63]))

In [ ]:
import re
import pandas as pd
import openai
import os
import json
from gpt3_sandbox.api.gpt import GPT
from gpt3_sandbox.api.gpt import Example
from pandasql import sqldf
from tqdm import tqdm
import numpy as np
import dotenv
config = dotenv.dotenv_values(".env")
openai.api_key = config['OPENAI_API_KEY_ms']

In [ ]:
prompt = \
"""
The database schema is as follows:
---
Table 'artists'
Columns: 'id', 'name'
Table 'sqlite_sequence'
Columns: 'name', 'seq'
Table 'albums'
Columns: 'id', 'title', 'artist_id'
Table 'employees'
Columns: 'id', 'last_name', 'first_name', 'title', 'reports_to', 'birth_date', 'hire_date', 'address', 'city', 'state', 'country', 'postal_code', 'phone', 'fax', 'email'
Table 'customers'
Columns: 'id', 'first_name', 'last_name', 'company', 'address', 'city', 'state', 'country', 'postal_code', 'phone', 'fax', 'email', 'support_rep_id'
Table 'genres'
Columns: 'id', 'name'
Table 'invoices'
Columns: 'id', 'customer_id', 'invoice_date', 'billing_address', 'billing_city', 'billing_state', 'billing_country', 'billing_postal_code', 'total'
Table 'media_types'
Columns: 'id', 'name'
Table 'tracks'
Columns: 'id', 'name', 'album_id', 'media_type_id', 'genre_id', 'composer', 'milliseconds', 'bytes', 'unit_price'
Table 'invoice_lines'
Columns: 'id', 'invoice_id', 'track_id', 'unit_price', 'quantity'
Table 'playlists'
Columns: 'id', 'name'
Table 'playlist_tracks'
Columns: 'playlist_id', 'track_id'
---

Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: "What is the count of customers that Steve Johnson supports?"
Generate the SQL step-by-step:
1. SELECT * FROM employees AS t1;
2. SELECT * FROM employees AS t1 JOIN customers AS t2 ON t2.support_rep_id = t1.id;
3. SELECT * FROM employees AS t1 JOIN customers AS t2 ON t2.support_rep_id = t1.id WHERE t1.first_name = "Steve" AND t1.last_name = "Johnson";
Therefore, the SQL query is ```SELECT * FROM employees AS t1 JOIN customers AS t2 ON t2.support_rep_id = t1.id WHERE t1.first_name = "Steve" AND t1.last_name = "Johnson";```.


The database schema is as follows:
---
Table 'stadium'
Columns: 'Stadium_ID', 'Location', 'Name', 'Capacity', 'Highest', 'Lowest', 'Average'
Table 'singer'
Columns: 'Singer_ID', 'Name', 'Country', 'Song_Name', 'Song_release_year', 'Age', 'Is_male'
Table 'concert'
Columns: 'concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year'
Table 'singer_in_concert'
Columns: 'concert_ID', 'Singer_ID'
---
Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: "Show the name and the release year of the song by the youngest singer.".
Generate the SQL step-by-step:"""


response = openai.Completion.create(engine='code-davinci-002',
                                            prompt=prompt,
                                            max_tokens=1024,
                                            temperature=0,
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop='```.', 
                                            frequency_penalty=0.7)
response

In [ ]:
prompt = \
"""
The database schema is as follows:
---
Table 'Addresses'
Columns: 'address_id', 'line_1_number_building', 'city', 'zip_postcode', 'state_province_county', 'country'
Table 'Staff'
Columns: 'staff_id', 'staff_address_id', 'nickname', 'first_name', 'middle_name', 'last_name', 'date_of_birth', 'date_joined_staff', 'date_left_staff'
Table 'Vehicles'
Columns: 'vehicle_id', 'vehicle_details'
Table 'Customers'
Columns: 'customer_id', 'customer_address_id', 'customer_status_code', 'date_became_customer', 'date_of_birth', 'first_name', 'last_name', 'amount_outstanding', 'email_address', 'phone_number', 'cell_mobile_phone_number'
Table 'Customer_Payments'
Columns: 'customer_id', 'datetime_payment', 'payment_method_code', 'amount_payment'
Table 'Lessons'
Columns: 'lesson_id', 'customer_id', 'lesson_status_code', 'staff_id', 'vehicle_id', 'lesson_date', 'lesson_time', 'price'
---

Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: 
"What is the total amount of moeny paid by the customer Carole Bernhard?".
SQL: ```
SELECT sum(t1.amount_payment)
FROM customer_payments AS t1
JOIN customers AS t2 ON t1.customer_id = t2.customer_id
WHERE t2.first_name = "Carole"
  AND t2.last_name = "Bernhard"
```


The database schema is as follows:
---
Table 'stadium'
Columns: 'Stadium_ID', 'Location', 'Name', 'Capacity', 'Highest', 'Lowest', 'Average'
Table 'singer'
Columns: 'Singer_ID', 'Name', 'Country', 'Song_Name', 'Song_release_year', 'Age', 'Is_male'
Table 'concert'
Columns: 'concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year'
Table 'singer_in_concert'
Columns: 'concert_ID', 'Singer_ID'
---

Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: 
"Show the name and the release year of the song by the youngest singer."
SQL:```"""


response = openai.Completion.create(engine='code-davinci-002',
                                            prompt=prompt,
                                            max_tokens=1024,
                                            temperature=0,
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop='```', 
                                            frequency_penalty=0.7)
response

In [ ]:
prompt = \
"""
The database schema is as follows:
---
Table 'Addresses'
Columns: 'address_id', 'line_1_number_building', 'city', 'zip_postcode', 'state_province_county', 'country'
Table 'Staff'
Columns: 'staff_id', 'staff_address_id', 'nickname', 'first_name', 'middle_name', 'last_name', 'date_of_birth', 'date_joined_staff', 'date_left_staff'
Table 'Vehicles'
Columns: 'vehicle_id', 'vehicle_details'
Table 'Customers'
Columns: 'customer_id', 'customer_address_id', 'customer_status_code', 'date_became_customer', 'date_of_birth', 'first_name', 'last_name', 'amount_outstanding', 'email_address', 'phone_number', 'cell_mobile_phone_number'
Table 'Customer_Payments'
Columns: 'customer_id', 'datetime_payment', 'payment_method_code', 'amount_payment'
Table 'Lessons'
Columns: 'lesson_id', 'customer_id', 'lesson_status_code', 'staff_id', 'vehicle_id', 'lesson_date', 'lesson_time', 'price'
---

Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: 
"What is the total amount of moeny paid by the customer Carole Bernhard?".
Generate the SQL step-by-step:
1. SELECT sum(t1.amount_payment) FROM customer_payments AS t1;
2. SELECT sum(t1.amount_payment) FROM customer_payments AS t1 
   JOIN customers AS t2 ON t1.customer_id = t2.customer_id;
3. SELECT sum(t1.amount_payment) FROM customer_payments AS t1 
   JOIN customers AS t2 ON t1.customer_id = t2.customer_id 
   WHERE t2.first_name = "Carole" AND t2.last_name = "Bernhard";
Therefore, the SQL query is:```
SELECT sum(t1.amount_payment)
FROM customer_payments AS t1
JOIN customers AS t2 ON t1.customer_id = t2.customer_id
WHERE t2.first_name = "Carole"
  AND t2.last_name = "Bernhard"
```.

The database schema is as follows:
---
Table 'stadium'
Columns: 'Stadium_ID', 'Location', 'Name', 'Capacity', 'Highest', 'Lowest', 'Average'
Table 'singer'
Columns: 'Singer_ID', 'Name', 'Country', 'Song_Name', 'Song_release_year', 'Age', 'Is_male'
Table 'concert'
Columns: 'concert_ID', 'concert_Name', 'Theme', 'Stadium_ID', 'Year'
Table 'singer_in_concert'
Columns: 'concert_ID', 'Singer_ID'
---

Generate a valid and syntactically correct SQL query that answers the following question and adheres to the schema listed above: 
"Show the name and the release year of the song by the youngest singer."
Generate the SQL step-by-step:
"""


response = openai.Completion.create(engine='code-davinci-002',
                                            prompt=prompt,
                                            max_tokens=1024,
                                            temperature=0,
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop='```.', 
                                            frequency_penalty=0.7)
response

In [177]:
prompt = \
"""
I'm a researcher in Database systems, and I am writing a conference paper. Help me improve the following sentences and highlight the changes.

Therefore, we use LIP and AJA to represent the simple but effective and commonly deployed techniques and perform comparisons with SOTA RL-based query optimizers. """

response = openai.Completion.create(engine='text-davinci-003',
                                            prompt=prompt,
                                            max_tokens=1024,
                                            temperature=0.5,
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            # stop='```.', 
                                            frequency_penalty=0)
print(response['choices'][0]['text'])



Therefore, we use LIP and AJA to represent the simple yet effective and commonly deployed techniques, and perform comparisons with state-of-the-art reinforcement learning-based query optimizers.


In [148]:
import pandas as pd
DF = pd.read_csv(f'./csv/203-csv/733.csv', sep=',')

In [149]:
DF

,Rank,Cyclist,Team,Time,UCI ProTour\nPoints
0,1,Alejandro Valverde (ESP),Caisse d'Epargne,"5h 29' 10\"",40""",NaN
1,2,Alexandr Kolobnev (RUS),Team CSC Saxo Bank,s.t.,30.0
2,3,Davide Rebellin (ITA),Gerolsteiner,s.t.,25.0
3,4,Paolo Bettini (ITA),Quick Step,s.t.,20.0
4,5,Franco Pellizotti (ITA),Liquigas,s.t.,15.0
5,6,Denis Menchov (RUS),Rabobank,s.t.,11.0
6,7,Samuel Sánchez (ESP),Euskaltel-Euskadi,s.t.,7.0
7,8,Stéphane Goubert (FRA),Ag2r-La Mondiale,"+ 2\"",5""",NaN
8,9,Haimar Zubeldia (ESP),Euskaltel-Euskadi,"+ 2\"",3""",NaN
9,10,David Moncoutié (FRA),Cofidis,"+ 2\"",1""",NaN


In [155]:
code = """
def get_country(s):
    return s.split(' ')[-1][1:-1]
DF['country'] = DF.apply(lambda x: get_country(x['Cyclist']), axis=1)"""
exec(code)

In [153]:
DF

,Rank,Cyclist,Team,Time,UCI ProTour\nPoints,country
0,1,Alejandro Valverde (ESP),Caisse d'Epargne,"5h 29' 10\"",40""",NaN,ESP
1,2,Alexandr Kolobnev (RUS),Team CSC Saxo Bank,s.t.,30.0,RUS
2,3,Davide Rebellin (ITA),Gerolsteiner,s.t.,25.0,ITA
3,4,Paolo Bettini (ITA),Quick Step,s.t.,20.0,ITA
4,5,Franco Pellizotti (ITA),Liquigas,s.t.,15.0,ITA
5,6,Denis Menchov (RUS),Rabobank,s.t.,11.0,RUS
6,7,Samuel Sánchez (ESP),Euskaltel-Euskadi,s.t.,7.0,ESP
7,8,Stéphane Goubert (FRA),Ag2r-La Mondiale,"+ 2\"",5""",NaN,FRA
8,9,Haimar Zubeldia (ESP),Euskaltel-Euskadi,"+ 2\"",3""",NaN,ESP
9,10,David Moncoutié (FRA),Cofidis,"+ 2\"",1""",NaN,FRA
